In [ ]:
(NOT EVEN A FIRST VERSION YET)

In [6]:
import random, collections, re
import wetsuite.datasets
import wetsuite.helpers.spacy

import spacy
from spacy.tokens.span import Span
import spacy.displacy
import wetsuite.helpers.spacy

In [7]:
dutch = spacy.load('nl_core_news_lg')

bwb_text = wetsuite.datasets.load('bwb-mostrecent-text')

## What named entities and does spacy know?

In [18]:
cherry_picked_xml = wetsuite.helpers.net.download( 'https://repository.officiele-overheidspublicaties.nl/bwb/BWBR0022604/2023-04-19_0/xml/BWBR0022604_2023-04-19_0.xml' )

spacy.displacy.render( dutch(bwb_text.data.get('BWBR0022604')) , style='ent', jupyter=True)

In [ ]:
# you could look at some more examples like:
for url, a_law_txt in bwb_text.data.random_sample( 1 ):
    display('-------------------------- %s -----------------------------'%url)
    doc = dutch( a_law_txt )
    
    # there is a nicer-looking entity visualizer...
    spacy.displacy.render(doc, style='ent', jupyter=True)    # Note: colab require explicit jupyter=True,  local notebooks do not.

    # ...but if we want to show multiple labelings at once, we need the 'span' visualizer, so we need to transplant entities to spans it picks up
    #myspans = []
    #for ent in doc.ents: 
    #    myspans.append( Span(doc, ent.start,ent.end, ent.label_) )
    #for nc in doc.noun_chunks:
    #    myspans.append( Span(doc,  nc.start,nc.end,  'nc') )
    #doc.spans["custom"] = myspans
    #spacy.displacy.render(doc, style="span", options={"spans_key": "custom"}, jupyter=True)

# which isn't bad, but also not as nice as we got in the cherry picked examples.



In [19]:

#spacy.require_gpu()
#english = spacy.load("en_core_web_trf")  

nlp = spacy.blank("en")  # "Build upon the spaCy Small Model" (seems wrong?)
ruler = nlp.add_pipe("entity_ruler") # adds EntityRuler to pipelines and returns reference

patterns = [
    {"label": "GPE", "pattern": "Treblinka"},
    {"label": "ORG", "pattern": "Wikipedia"},

    {"label": "TERM", "pattern": "depot"},
    {"label": "F", "pattern": "vehicle maintenance facilit.*"},
]

ruler.add_patterns(patterns)

# test on example sentence
text = "The term depot is not used in reference to vehicle maintenance facilities in the U.S"
doc = nlp(text)
for ent in doc.ents:
    print (ent.text, ent.label_)

depot TERM


In [9]:
from spacy.tokens import Doc
from spacy.training import Example

predicted = Doc(nlp.vocab, words=["Apply", "some", "sunscreen"])
token_ref = ["Apply", "some", "sun", "screen"]
tags_ref = ["VERB", "DET", "NOUN", "NOUN"]
example = Example.from_dict(predicted, {"words": token_ref, "tags": tags_ref})
print( 'predicted', example.predicted )
print( 'reference', example.reference )

predicted Apply some sunscreen 
reference Apply some sun screen 


In [20]:

count = collections.defaultdict(int)

for plaintext, in random.sample( bwb_rows, 10):

    for paragraph in re.split( '\n{2,}', plaintext ):
        doc = english( paragraph )
        #print( paragraph )
        for chunk in doc.noun_chunks:
            #print( 'NC[%r]'%chunk )
            count[chunk.text] += 1

it = list(count.items())
it.sort(key = lambda x:x[1])
for s, cnt in it:
    print('%s\t%s'%(cnt,s))

NameError: name 'bwb_rows' is not defined